# **Đồ án cuối kì**
### **Mã lớp:** CS114.N21

### **Giảng viên:** Phạm Nguyễn Trường An
### **Đề tài:** Phân loại đánh giá của khách hàng trên Shopee

### **Thành viên nhóm H2O:**
- Nguyễn Quốc Huy Hoàng - 20520051
- Ngô Quang Vinh - 19522523

Import các thư viện cần thiết

In [22]:
import numpy as np
import pandas as pd
import json
import requests
import re
from pprint import pprint
import torch
import underthesea # Thư viện tách từ
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from joblib import dump

## Xử lý dữ liệu

### Đọc dữ liệu sau đó chia làm 2 tập train/test

In [23]:
# Đọc dữ liệu từ file Excel
file_path = "dataset.xlsx"
data_frame = pd.read_excel(file_path, header=None)

# Truy cập vào cột thứ nhất và thứ hai
labels = data_frame.iloc[:, 0].tolist()
reviews = data_frame.iloc[:, 1].tolist()

X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=20520051, shuffle=True)
y_train = np.array(y_train)
y_test = np.array(y_test)
pprint("X_train[0]")
pprint(X_train[0])

'X_train[0]'
'Kêu 1 cặp 2 cái đặt x5 mà giao đc 5 cái shop làm ăn kiểu gì v'


In [24]:
tichcuc = "sản phẩm tốt chất lượng tốt "
tieucuc = "sản phẩm tệ kém chất lượng "
with open('negative icon.txt', 'r', encoding='utf-8') as file:
    icon_tieu_cuc = file.read()
print(icon_tieu_cuc)
with open('positive_icon.txt', 'r', encoding='utf-8') as file:
    icon_tich_cuc = file.read()
print(icon_tich_cuc)

🙂👿😠😡🙁😒😫😕😩😣😟😭😢😖😔😞☹️🤧🤒😷🤕😵🤢🤠🤡👹👺👻💀👽😰😨😧🤥😈🙃👎🤬👋😌😏😬💩😓😤😮‍💨😑
😀😊😉😍😘😗😙🤗😚😛😝😜😋🤑😎😇♥️❤️💛💚💙💜🖤💖💝🤩🥰😅💞💋🤗😋🥳



### Chuẩn hóa dữ liệu

In [35]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Icon biểu tượng
    # for val in row:
    #     if val in icon_tich_cuc:
    #         row = tichcuc + row
    #     if val in icon_tieu_cuc:
    #         row = tieucuc + row
    # Xoá hết những cái không phải chữ và số
    row =  re.sub(r"[^\w\s]", " ", row)

    # # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    # row = re.sub(r"[\.,\?]+$-", "", row)
    # # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    # row = row.replace(",", " ").replace(".", " ") \
    #     .replace(";", " ").replace("“", " ") \
    #     .replace(":", " ").replace("”", " ") \
    #     .replace('"', " ").replace("'", " ") \
    #     .replace("!", " ").replace("?", " ") \
    #     .replace("-", " ").replace("?", " ")
    # # Nếu có nhiều khoảng trắng thì rút gọn còn 1
    row = row.replace('\n'," ")
    row = re.sub(r"\s+", " ", row)
    # Đưa hết về chữ thường
    row = row.strip().lower()
    str = ""
    str += row[0]
    for i in range(1, len(row)):
        if row[i] != row[i - 1]:
            str += row[i]
    # pprint(str)
    # print(ascii(row[16]))
    # Chuyển thành dạng không dấu
    # str = unidecode(str)
    return str

### Lấy danh sách stop word của tiếng Việt

In [26]:
# Hàm load danh sách các từ vô nghĩa: lắm, ạ, à, bị, vì..
sw = []
with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    sw.append(line.replace("\n",""))

### Chuyển văn bảng thành dạng vetor đặc trưng

In [27]:
list_model_trans = []
list_tokenizer = []
list_name_trans = []

list_model_trans.append(AutoModel.from_pretrained("Fsoft-AIC/videberta-xsmall"))
list_tokenizer.append(AutoTokenizer.from_pretrained("Fsoft-AIC/videberta-xsmall"))
list_name_trans.append('videberta-xsmall')

list_model_trans.append(AutoModel.from_pretrained("Fsoft-AIC/videberta-base"))
list_tokenizer.append(AutoTokenizer.from_pretrained("Fsoft-AIC/videberta-base"))
list_name_trans.append('videberta-base')

list_model_trans.append(AutoModel.from_pretrained("vinai/phobert-base"))
list_tokenizer.append(AutoTokenizer.from_pretrained("vinai/phobert-base"))
list_name_trans.append('phobert-base')

list_model_trans.append(AutoModel.from_pretrained("vinai/phobert-large"))
list_tokenizer.append(AutoTokenizer.from_pretrained("vinai/phobert-large"))
list_name_trans.append('phobert-large')

list_model_trans.append(AutoModel.from_pretrained("vinai/phobert-base-v2"))
list_tokenizer.append(AutoTokenizer.from_pretrained("vinai/phobert-base-v2"))
list_name_trans.append('phobert-base-v2')

Some weights of the model checkpoint at Fsoft-AIC/videberta-xsmall were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at Fsoft-AIC/videberta-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predicti

In [36]:
# Hàm tạo ra bert features
def make_bert_features(v_text, model_trans, tokenizer):
    v_tokenized = []
    max_len = 150 # Mỗi câu dài tối đa 200 từ
    for i_text in v_text:
        # print("Đang xử lý line = ", i_text)
        # Chuẩn hóa

        i_text = i_text.replace("\n"," ")
        i_text = standardize_data(i_text)

        # # Phân thành từng từ
        # line = underthesea.word_tokenize(i_text)

        # # Lọc các từ vô nghĩa
        # # filtered_sentence = [w for w in line if not w in sw]
        # filtered_sentence = line

        # # Ghép lại thành câu như cũ sau khi lọc
        # line = " ".join(filtered_sentence[:150])
        # line = underthesea.word_tokenize(line, format="text")

        line = ViTokenizer.tokenize(i_text)[:150]


        # print("Word segment  = ", line)
        # Tokenize bởi BERT
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = []
    # Chèn thêm số 1 vào cuối câu nếu như không đủ từ hoặc xóa nếu dư
    for line in v_tokenized:
        if len(line) < max_len:
            padded.append(line + [1] * (max_len - len(line)))
        else:
            padded.append(line[: max_len])
    padded = np.array(padded)

    # print('padded:', padded[0])
    # print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = np.where(padded == 1, 0, 1)
    # print('attention mask:', attention_mask[0])

    # Chuyển thành tensor
    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)

    # Lấy features dầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = model_trans(input_ids= padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    print(v_features.shape)
    v_features = np.array(v_features)
    return v_features

## Mô hình phân lớp


In [29]:
def solve(model, parameters, X_train, y_train, X_test, y_test):
    # Grid search để tìm bộ siêu tham số tốt nhất
    clf = GridSearchCV(model, param_grid=parameters,  cv = 5, scoring='f1', n_jobs=-1)
    grid_search = clf.fit(X_train, y_train)

    # In ra bộ siêu tham số tìm được
    print('best prarams:', clf.best_params_)

    # Dùng model tốt nhất tìm được để dự đoán
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Đánh giá mô hình
    f1 = f1_score(y_test, y_pred)
    print("Model: "+ type(model).__name__+",", "F1 score:", f1)
    
    # Save model
    dump(best_model, type(model).__name__+'.joblib')

    return f1

In [30]:
model_config = []

### SVM

In [31]:
model_config.append(
    (SVC(),{
        'kernel': ('linear', 'rbf'),
        'C': [0.5, 1, 2, 4],
        'gamma': ['scale']#, 0.125, 0.25, 0.5, 1, 2, 4]
    }))

### Logistic Regression

In [32]:
model_config.append(
    (LogisticRegression(),{
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2'],
        'solver': ['lbfgs', 'liblinear', 'saga']
    }))

### Radom Forest

In [33]:
model_config.append(
    (RandomForestClassifier(),{
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10]
    }))

## Chạy thực nghiệm

In [34]:
df = {}
for i in range(len(list_model_trans)):
    model_trans = list_model_trans[i]
    tokenizer = list_tokenizer[i]
    name_trans = list_name_trans[i]

    # Chuyển văn bản thành vector đặc trưng
    new_X_train = make_bert_features(X_train, model_trans, tokenizer)
    new_X_test = make_bert_features(X_test, model_trans, tokenizer)

    value_f1 = []
    for model, params in model_config:
        f1 = solve(model, params, new_X_train, y_train, new_X_test, y_test)
        value_f1.append(f1)
        # break
    df[name_trans] = value_f1
    # break
df = pd.DataFrame(df)
df


Padd =  torch.Size([1948, 150])
(1948, 384)
Padd =  torch.Size([487, 150])
(487, 384)
best prarams: {'C': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
Model: SVC, F1 score: 0.7334200260078023


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

best prarams: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Model: LogisticRegression, F1 score: 0.7334200260078023
best prarams: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Model: RandomForestClassifier, F1 score: 0.7334200260078023
Padd =  torch.Size([1948, 150])
(1948, 768)
Padd =  torch.Size([487, 150])
(487, 768)
best prarams: {'C': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
Model: SVC, F1 score: 0.7334200260078023


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

best prarams: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Model: LogisticRegression, F1 score: 0.7334200260078023
best prarams: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Model: RandomForestClassifier, F1 score: 0.7334200260078023
Padd =  torch.Size([1948, 150])
(1948, 768)
Padd =  torch.Size([487, 150])
(487, 768)
best prarams: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Model: SVC, F1 score: 0.9324090121317158


...

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

best prarams: {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
Model: LogisticRegression, F1 score: 0.9148936170212766
best prarams: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Model: RandomForestClassifier, F1 score: 0.8797250859106529
Padd =  torch.Size([1948, 150])
(1948, 768)
Padd =  torch.Size([487, 150])
(487, 768)
best prarams: {'C': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Model: SVC, F1 score: 0.9368421052631578


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

best prarams: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Model: LogisticRegression, F1 score: 0.9368421052631578
best prarams: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Model: RandomForestClassifier, F1 score: 0.910321489001692


,videberta-xsmall,videberta-base,phobert-base,phobert-large,phobert-base-v2
0,0.73342,0.73342,0.932409,0.917108,0.936842
1,0.73342,0.73342,0.930796,0.914894,0.936842
2,0.73342,0.73342,0.913413,0.879725,0.910321
